In [1]:
import pandas as pd
import numpy as np

In [3]:
import pickle
with open("data/df.pkl", "rb") as f:
    df = pickle.load(f)

df.head()           

,Company,TypeName,Ram,Weight,Price,Touchscreen,IPS,ppi,CPU_brand,Gpu_brand,os,HDD,SDD
0,Apple,Ultrabook,8,1.37,71378.6832,0,1,226.983005,Intel Core i5,Intel,MAc,0,128
1,Apple,Ultrabook,8,1.34,47895.5232,0,0,127.677940,Intel Core i5,Intel,MAc,0,0
2,HP,Notebook,8,1.86,30636.0000,0,0,141.211998,Intel Core i5,Intel,Other/No OS/Linux,0,256
3,Apple,Ultrabook,16,1.83,135195.3360,0,1,220.534624,Intel Core i7,AMD,MAc,0,512
4,Apple,Ultrabook,8,1.37,96095.8080,0,1,226.983005,Intel Core i5,Intel,MAc,0,256


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [5]:
x=df.drop(columns=["Price"])
y=np.log(df["Price"])

In [6]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.15,random_state=2)

In [7]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

In [8]:
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor,AdaBoostRegressor,ExtraTreesRegressor
from sklearn.ensemble import BaggingRegressor


In [20]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [27]:
models = {
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(alpha=0.001),
    "Ridge": Ridge(alpha=10),
    "K-Neighbors Regressor": KNeighborsRegressor(n_neighbors=4),
    "Decision Tree": DecisionTreeRegressor(max_depth=8),
    "Random Forest Regressor": RandomForestRegressor(n_estimators=200,max_features="sqrt",max_depth=20,min_samples_leaf=1,min_samples_split=2),
    "ExtraTreesRegressor": ExtraTreesRegressor(n_estimators=100,max_features="sqrt",max_depth=20,min_samples_leaf=1,min_samples_split=2),
    "AdaBoost Regressor": AdaBoostRegressor(n_estimators=15,learning_rate=1.0),
    "GradientBoostingRegressor":GradientBoostingRegressor(n_estimators=500)
}

In [28]:
model_list = []
r2_list =[]
step1=ColumnTransformer(transformers=[("col_tnf",OneHotEncoder(sparse_output=False,drop="first"),[0,1,7,8,9])],remainder="passthrough")

for i in range(len(list(models))):
    model = list(models.values())[i]
    pipe=Pipeline([("step1",step1),("step2",model)])
    pipe.fit(x_train,y_train)

    # Make predictions
    y_train_pred = pipe.predict(x_train)
    y_test_pred = pipe.predict(x_test)
    
    # Evaluate Train and Test dataset
    model_train_mae , model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)

    model_test_mae , model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    r2_list.append(model_test_r2)
    
    print('='*35)
    print('\n')

Linear Regression
Model performance for Training set
- Root Mean Squared Error: 0.2545
- Mean Absolute Error: 0.1985
- R2 Score: 0.8340
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 0.2715
- Mean Absolute Error: 0.2102
- R2 Score: 0.8073


Lasso
Model performance for Training set
- Root Mean Squared Error: 0.2612
- Mean Absolute Error: 0.2048
- R2 Score: 0.8251
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 0.2716
- Mean Absolute Error: 0.2111
- R2 Score: 0.8072


Ridge
Model performance for Training set
- Root Mean Squared Error: 0.2617
- Mean Absolute Error: 0.2045
- R2 Score: 0.8244
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 0.2677
- Mean Absolute Error: 0.2093
- R2 Score: 0.8127


K-Neighbors Regressor
Model performance for Training set
- Root Mean Squared Error: 0.2189
- Mean Absolute Error: 0.1612
- R2 Score: 0.8771
-----------------------

In [29]:
pd.DataFrame(list(zip(model_list, r2_list)), columns=['Model Name', 'R2_Score']).sort_values(by=["R2_Score"],ascending=False)


,Model Name,R2_Score
5,Random Forest Regressor,0.903916
6,ExtraTreesRegressor,0.895215
8,GradientBoostingRegressor,0.884473
4,Decision Tree,0.840272
2,Ridge,0.812733
0,Linear Regression,0.807328
1,Lasso,0.807185
3,K-Neighbors Regressor,0.797152
7,AdaBoost Regressor,0.791881


In [31]:
step1=ColumnTransformer(transformers=[("col_tnf",OneHotEncoder(sparse_output=False,drop="first"),[0,1,7,8,9])],remainder="passthrough")
step2=RandomForestRegressor(n_estimators=200,max_features="sqrt",max_depth=20,min_samples_leaf=1,min_samples_split=2)
Pipeline([("step1",step1),("step2",step2)])
pipe=Pipeline([("step1",step1),("step2",step2)])
pipe.fit(x_train,y_train)
y_predit=pipe.predict(x_test)
print(r2_score(y_test,y_predit))
print(mean_absolute_error(y_test,y_predit))
print(pipe.score(x_test,y_test)*100)
print(pipe.score(x_train,y_train))

0.902982659528887
0.14519909418537946
90.2982659528887
0.9806503506757843


In [14]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import r2_score, mean_absolute_error

param_grid = {
    "n_estimators": [200, 400, 600, 800, 1200],
    "max_depth": [10, 20, 30, 40, 50, None],
    "min_samples_split": [2, 4, 6, 10],
    "min_samples_leaf": [1, 2, 4, 8],
    "max_features": ["sqrt", "log2",None],
    "bootstrap": [True, False]
}

rf = RandomForestRegressor(random_state=42)

random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_grid,
    n_iter=20,
    scoring='r2',
    cv=5,
    verbose=2,
    n_jobs=-1,
    random_state=42
)

# Fit
random_search.fit(x_train, y_train)

# Best model
best_model = random_search.best_estimator_
print("Best Params:", random_search.best_params_)

# Predictions
y_pred = best_model.predict(x_test)

print("R2 Score:", r2_score(y_test, y_pred))
print("MAE:", mean_absolute_error(y_test, y_pred))


Fitting 5 folds for each of 20 candidates, totalling 100 fits


ValueError: 
All the 100 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "e:\ml_project\venv\lib\site-packages\sklearn\model_selection\_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "e:\ml_project\venv\lib\site-packages\sklearn\base.py", line 1152, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "e:\ml_project\venv\lib\site-packages\sklearn\ensemble\_forest.py", line 348, in fit
    X, y = self._validate_data(
  File "e:\ml_project\venv\lib\site-packages\sklearn\base.py", line 622, in _validate_data
    X, y = check_X_y(X, y, **check_params)
  File "e:\ml_project\venv\lib\site-packages\sklearn\utils\validation.py", line 1146, in 